In [10]:
# authorization.py
import tekore as tk

def auth():
  client_id  = "b8827e19044d482d9cb1da2cd0d2346f"
  secret_key = "02759ef505ba4d8aafa72242ea1d6667"
  redirect_uri = 'https://musica.com/callback'
  token = tk.request_client_token(client_id, secret_key)
  return tk.Spotify(token)

In [ ]:
#I am going to use a method that draws 100 tracks from over 100 genres, 
#resulting in a track database of around 10000 tracks with music from various styles.
import sys
from auth import Authorization
import pandas as pd
import time
import spotify
from tqdm import tqdm

# authorize and retrieve Spotify object
spotify = Authorization.auth() # type: ignore

# get genre seeds
genres = spotify.recommendation_genre_seeds() 

data_dict = {
  "id":[], 
  "genre":[], 
  "track_name":[], 
  "artist_name":[],
  "valence":[],     # <-- this is our psychological value
  "energy":[]       # <-- this too 
}
  
# Get recommendation for each genre
for genre in tqdm(genres):
      
  recs = spotify.recommendations(seed_genres = [genre], limit = 100)
  recs = recs["tracks"]
  
  for track in recs:
      data_dict["id"].append(track["id"])
      data_dict["genre"].append(genre)
      track_meta = spotify.track(track["id"])
      data_dict["track_name"].append(track_meta["name"])
      data_dict["artist_name"].append(track_meta["album"]["artists"][0]["name"])
      track_features = spotify.audio_features(track["id"])
      data_dict["valence"].append(track_features[0]["valence"])
      data_dict["energy"].append(track_features[0]["energy"])

# Store data in pandas DataFrame
df = pd.DataFrame(data_dict)

# Drop duplicates
df.drop_duplicates(subset = "id", keep = "first", inplace = True)
df.to_csv("arousal_dataset.csv", index = False)

In [13]:
def distance(p1, p2):
    distance_x = p2[0]-p1[0]
    distance_y = p2[1]-p1[1]
    distance_vec = [distance_x, distance_y]
    norm = (distance_vec[0]**2 + distance_vec[1]**2)**(1/2)
    return norm

In [ ]:
import pandas as p
import random
import auth
import numpy as np
#from numpy.linalg import normd
from numpy.linalg import norm

#read dataset from which was created above
df = pd.read_csv("arousal_dataset.csv")

#lets combine valence and energy 
df["mood_vec"] = df[["valence", "energy"]].values.tolist()

In [ ]:
def recommend(track_id, ref_df, spotify, n_recs = 5):
    features = spotify.track_audio_features(track_id)
    vector = np.array([features.valence, features.energy])
    
    ref_df["distances"] = ref_df["mood_vec"].apply(lambda x: norm(vector-np.array(x)))

    df_sorted = ref_df.sort_values(by = "distances", ascending = True)

    df_sorted = ref_df_sorted[ref_df_sorted["id"] != track_id]
    
    return ref_df_sorted.iloc[:n_recs]

In [ ]:
track = "4rDbp1vnvEhieiccprPMdI"
recommend(track_id = track, ref_df = df, sp = sp, n_recs = 5)